In [129]:
import librosa
import numpy as np
from pydub import AudioSegment
from sklearn import preprocessing
from matplotlib import pyplot as plt
from scipy import signal
import pandas as pd

In [130]:
pathAudio = 'C:/Users/Admin/Downloads/Technology Lab/sampling/in-1sec/'
files = librosa.util.find_files(pathAudio, ext=['wav'])
files = sorted(files, key=lambda x: int(x.split('_')[-1].split(".")[0]))
samples = []
for file_name in files:
    loaded_audio_file = AudioSegment.from_wav(file_name)
    numeric_sample_array = loaded_audio_file.get_array_of_samples()
    resampled_part = signal.resample(numeric_sample_array, 882)
    samples.append(resampled_part)
samples = np.array(samples)
samples.shape

(222, 882)

In [131]:
min_max_scaler = preprocessing.MinMaxScaler()
normalized_samples = min_max_scaler.fit_transform(samples)
normalized_samples.shape

(222, 882)

In [132]:
normalized_samples = normalized_samples.reshape(222, 882, 1)
normalized_samples.shape

(222, 882, 1)

In [133]:
all_samples = normalized_samples.reshape(195804)
all_samples

array([0.61667124, 0.47142056, 0.44955661, ..., 0.47062554, 0.49467703,
       0.54357459])

In [134]:
frami = pd.DataFrame(columns=['actual'], data=all_samples)
frami

,actual
0,0.616671
1,0.471421
2,0.449557
3,0.553337
4,0.479708
...,...
195799,0.515698
195800,0.447553
195801,0.470626
195802,0.494677


In [135]:
X = []
Y = []
for i in range(0, len(all_samples)-880):
    if i < len(all_samples)-900:
        Y.append(all_samples[i+880:i+900])
        X.append(np.array(all_samples[i:i+880]))

In [189]:
X = np.array(X).reshape(-1, 880, 1)
Y = np.array(Y).reshape(194904, -1)

In [190]:
X.shape

(194904, 880, 1)

In [191]:
Y.shape

(194904, 20)

In [192]:
from keras.layers import LSTM, Dense
from keras.models import  Sequential

model = Sequential()
model.add(LSTM(100, input_shape=(880, 1), return_sequences=True, stateful=True, batch_size=32, activation='sigmoid'))
model.add(Dense(20))

model.compile(optimizer='rmsprop', loss='mse')

In [193]:
model.fit(X, Y, epochs=10)

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'gradient_tape/mean_squared_error/BroadcastGradientArgs' defined at (most recent call last):
    File "C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "D:\Program Files\JetBrains\PycharmProjects\deeplearning-course\venv\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "D:\Program Files\JetBrains\PycharmProjects\deeplearning-course\venv\lib\site-packages\traitlets\config\application.py", line 976, in launch_instance
      app.start()
    File "D:\Program Files\JetBrains\PycharmProjects\deeplearning-course\venv\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "D:\Program Files\JetBrain